# **Prediction**

## **Data Loading**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import numpy as np

dat_path = '/content/drive/MyDrive/harvard/bst210/final/pred_dat.csv'
dat = pd.read_csv(dat_path,encoding='latin-1')

In [ ]:
dat

,FIPS Code,State,county_name,"Less than a high school diploma, 2016-20","High school diploma only, 2016-20","Some college or associate's degree, 2016-20","Bachelor's degree or higher, 2016-20","Percent of adults with less than a high school diploma, 2016-20","Percent of adults with a high school diploma only, 2016-20","Percent of adults completing some college or associate's degree, 2016-20",...,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,state,State Capital,Region
0,1001,AL,Autauga County,4273,11880,10986,10721,11.286318,31.378763,29.017433,...,14.3,77.399902,18.679488,0.967023,0.925373,56.069818,5.872131,Alabama,Montgomery,South
1,1003,AL,Baldwin County,14823,42272,48832,49636,9.528615,27.173557,31.390497,...,17.5,86.431496,9.601734,0.669841,0.958380,53.935010,5.381478,Alabama,Montgomery,South
2,1005,AL,Barbour County,4497,6361,4872,2067,25.268303,35.741978,27.375401,...,26.5,47.363731,46.765929,0.905985,3.445715,43.276946,6.266977,Alabama,Montgomery,South
3,1007,AL,Bibb County,3056,7206,3911,1814,19.115532,45.074123,24.463627,...,22.7,76.654574,21.438683,0.092904,0.030968,57.335990,4.179861,Alabama,Montgomery,South
4,1009,AL,Blount County,6838,13975,13725,5276,17.174864,35.100719,34.472797,...,19.0,95.097903,1.531797,0.143823,0.961705,59.439854,6.894123,Alabama,Montgomery,South
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,56037,WY,Sweetwater County,2098,9115,11034,5989,7.430231,32.281487,39.077774,...,11.3,92.133476,0.866613,0.857679,1.684088,54.385755,8.929776,Wyoming,Cheyenne,West
3043,56039,WY,Teton County,945,2778,4270,9789,5.314363,15.622540,24.013046,...,5.5,93.160325,0.322711,1.900408,2.783380,50.580188,1.745232,Wyoming,Cheyenne,West
3044,56041,WY,Uinta County,903,4931,4758,2501,6.896815,37.661346,36.340031,...,13.0,94.233158,0.195891,0.119446,1.538462,57.451346,6.918372,Wyoming,Cheyenne,West
3045,56043,WY,Washakie County,441,1587,2174,1313,7.996374,28.776066,39.419765,...,13.9,91.238095,0.678571,0.166667,4.142857,51.708428,5.370370,Wyoming,Cheyenne,West


## **Data Preprocessing**

In [ ]:
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

def tvt(X, y):
    """
    Perform a 80/20 train-val-test split and return in dictionary
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    return {'X_train':X_train, 'X_test': X_test, 
            'y_train': np.array(y_train), 'y_test':np.array(y_test)}

# clean up data, drop redundant columns
y = dat['TARGET_deathRate']
# don't need county_name since its for obs
X = dat.drop(
    ['TARGET_deathRate', 'State Capital', 'state', 
     'FIPS Code', 'Geography', 'county_name', 'PctSomeCol18_24'],  
    axis = 1)
X = pd.concat([X, pd.get_dummies(dat.State), 
           pd.get_dummies(dat.Region)], 
           axis = 1)
# drop text and use median income instead of binned b/c probably sample
X = X.drop(['State', 'Region', 'binnedInc'], axis = 1)

# 1. imputation
simpute = SimpleImputer(strategy='median').fit(X) # median b/c potential skew
X_sp = simpute.transform(X)
X_sp = tvt(X_sp, y)

mulpute = IterativeImputer(max_iter=10, random_state=0, verbose = 2).fit(X)
X_mp = mulpute.transform(X)
X_mp = tvt(X_mp, y)

knnpute = KNNImputer(n_neighbors=5, weights="uniform").fit(X)
X_kp = knnpute.transform(X)
X_kp = tvt(X_kp, y)

missing_idx = [x[1] for x in list(X.unstack()[X.isnull().unstack()].index)]
X_dp = X.drop(index=  missing_idx)
X_dp = tvt(X_dp, y.drop(index = missing_idx))

# 2. standardize
scaler_sp = pp.StandardScaler().fit(X_sp['X_train'])  # get mean and sd
X_sp['X_train'] = scaler_sp.transform(X_sp['X_train'])

scaler_mp = pp.StandardScaler().fit(X_mp['X_train'])  # get mean and sd
X_mp['X_train'] = scaler_mp.transform(X_mp['X_train'])

scaler_kp = pp.StandardScaler().fit(X_kp['X_train'])  # get mean and sd
X_kp['X_train'] = scaler_kp.transform(X_kp['X_train'])

scaler_dp = pp.StandardScaler().fit(X_dp['X_train'])
X_dp['X_train'] = scaler_dp.transform(X_dp['X_train'])


[IterativeImputer] Completing matrix with shape (3047, 94)
[IterativeImputer] Ending imputation round 1/10, elapsed time 7.68
[IterativeImputer] Change: 39.65345861336172, scaled tolerance: 10170.292 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (3047, 94)
[IterativeImputer] Ending imputation round 1/1, elapsed time 0.01


## **Modeling**

### **Ridge**

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error

def model_metrics(y_pred, y):
    return pd.DataFrame(
        {
            "RMSE": [np.sqrt(mean_squared_error(y_pred, y))],
            "MedAE": [median_absolute_error(y_pred, y)],
            "R2": [r2_score(y_pred, y)]
        }
    )

# for various imputes do 20-fold CV ridge fits (reg=(0.1, 1.0, 10.0))
ridge_sp = RidgeCV(cv = 20)
ridge_sp.fit(X_sp['X_train'], X_sp['y_train'])
y_sp_pred = ridge_sp.predict(scaler_sp.transform(X_sp['X_test']))
print('Median Impute: ', model_metrics(y_sp_pred, X_sp['y_test']))


ridge_mp = RidgeCV(cv = 20)
ridge_mp.fit(X_mp['X_train'], X_mp['y_train'])
y_mp_pred = ridge_mp.predict(scaler_mp.transform(X_mp['X_test']))
print('Multiple Impute: ', model_metrics(y_mp_pred,X_mp['y_test']))

ridge_kp = RidgeCV(cv = 20)
ridge_kp.fit(X_kp['X_train'], X_kp['y_train'])
y_kp_pred = ridge_kp.predict(scaler_kp.transform(X_kp['X_test']))
print('KNN Impute: ', model_metrics(y_kp_pred, X_kp['y_test']))

ridge_dp = RidgeCV(cv = 20)
ridge_dp.fit(X_dp['X_train'], X_dp['y_train'])
y_dp_pred = ridge_dp.predict(scaler_dp.transform(X_dp['X_test']))
print('Complete Cases: ', model_metrics(y_dp_pred, X_dp['y_test']))

Median Impute:          RMSE      MedAE        R2
0  17.851854  10.523669  0.278284
Multiple Impute:          RMSE     MedAE        R2
0  16.813691  9.724589  0.304927
KNN Impute:          RMSE     MedAE        R2
0  16.780724  8.938034  0.364457
Complete Cases:          RMSE      MedAE        R2
0  19.138594  10.287969  0.243996


### **Gradient Boosting Regression Trees**

In [ ]:
import xgboost as xg
from sklearn.model_selection import GridSearchCV

# initial param set, will narrow down since all datasets are similar
params = {'n_estimators':[150], 
         'max_depth':[3, 6],  # XGB complexity
         'min_child_weight':[1, 5, 10], # min sum of obs req in a child
         'learning_rate':[0.03, 0.1, 0.2, 0.3], # step size shrinkage
         'gamma':[0, 1, 10, 100], # complexity penalty term
         'subsample':[0.5, 1], # subsample for each tree
         'objective':['reg:squarederror', 'reg:squaredlogerror']}

xgb_sp = xg.XGBRegressor(seed = 0)
gs_sp = GridSearchCV(xgb_sp, params, cv = 5)

gs_sp.fit(X_sp['X_train'], X_sp['y_train'])
print('Median Impute Best Params: ', gs_sp.best_params_)
y_pred = gs_sp.predict(scaler_sp.transform(X_sp['X_test']))
print('Median Impute: ', model_metrics(y_sp_pred, X_sp['y_test']))

In [88]:
# using best params
params = {'gamma': 1, 'learning_rate': 0.3, 'max_depth': 3, 
          'min_child_weight': 1, 'n_estimators': 150, 
          'objective': 'reg:squarederror', 
          'subsample': 1,
          'seed' : 0} 

xgb_sp = xg.XGBRegressor(**params).fit(X_sp['X_train'], X_sp['y_train'])
xgb_sp.fit(X_sp['X_train'], X_sp['y_train'])
y_sp_pred = xgb_sp.predict(scaler_sp.transform(X_sp['X_test']))
print('Median Impute: ', model_metrics(y_sp_pred, X_sp['y_test']))

xgb_mp = xg.XGBRegressor(**params).fit(X_mp['X_train'], X_mp['y_train'])
xgb_mp.fit(X_mp['X_train'], X_mp['y_train'])
y_mp_pred = xgb_mp.predict(scaler_mp.transform(X_mp['X_test']))
print('Multiple Impute: ', model_metrics(y_mp_pred, X_mp['y_test']))

xgb_kp = xg.XGBRegressor(**params).fit(X_kp['X_train'], X_kp['y_train'])
xgb_kp.fit(X_kp['X_train'], X_kp['y_train'])
y_kp_pred = xgb_kp.predict(scaler_kp.transform(X_kp['X_test']))
print('KNN Impute: ', model_metrics(y_kp_pred, X_kp['y_test']))

xgb_dp = xg.XGBRegressor(**params).fit(X_dp['X_train'], X_dp['y_train'])
xgb_dp.fit(X_dp['X_train'], X_dp['y_train'])
y_dp_pred = xgb_dp.predict(scaler_dp.transform(X_dp['X_test']))
print('Complete Cases: ', model_metrics(y_dp_pred, X_dp['y_test']))

Median Impute:          RMSE     MedAE        R2
0  15.336701  9.214442  0.539073
Multiple Impute:         RMSE     MedAE        R2
0  14.86273  8.510454  0.531607
KNN Impute:          RMSE     MedAE        R2
0  14.851196  8.985658  0.541731
Complete Cases:          RMSE      MedAE       R2
0  17.357475  10.198532  0.41381
